In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration


/opt/conda/envs/llmz310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Check HF

- Default HF download directory : HF_HOME
- Find another path where there are some available disk space

In [ ]:
!df -h

- Change dicrectory

In [ ]:
import os
os.environ['HF_HOME'] = '/run/cache/'

In [2]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl", device_map="auto")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


RuntimeError: unable to mmap 9449619912 bytes from file </home/codespace/.cache/huggingface/hub/models--google--flan-t5-xl/snapshots/7d6315df2c2fb742f0f5b556879d730926ca9001/model-00001-of-00002.safetensors>: Cannot allocate memory (12)